In [7]:
import functools
import json
import math
import pandas as pd
import pennylane as qml
import pennylane.numpy as np
import scipy

def generator_info(operator):
    """Provides the generator of a given operator.

    Args:
        operator (qml.ops): A PennyLane operator

    Returns:
        (qml.ops): The generator of the operator.
        (float): The coefficient of the generator.
    """
    gen = qml.generator(operator, format="observable")
    return gen.ops[0], gen.coeffs[0]


def derivative(op_order, params, diff_idx, wires, measured_wire):
    """A function that calculates the derivative of a circuit w.r.t. one parameter.

    NOTE: you cannot use qml.grad in this function.

    Args:
        op_order (list(int)):
            This is a list of integers that defines the circuit in question.
            The entries of this list correspond to dictionary keys to op_dict.
            For example, [1,0,2] means that the circuit in question contains
            an RY gate, an RX gate, and an RZ gate in that order.

        params (np.array(float)):
            The parameters that define the gates in the circuit. In this case,
            they're all rotation angles.

        diff_idx (int):
            The index of the gate in the circuit that is to be differentiated
            with respect to. For instance, if diff_idx = 2, then the derivative
            of the third gate in the circuit will be calculated.

        wires (list(int)):
            A list of wires that each gate in the circuit will be applied to.

        measured_wire (int):
            The expectation value that needs to be calculated is with respect
            to the Pauli Z operator. measured_wire defines what wire we're
            measuring on.

    Returns:
        float: The derivative evaluated at the given parameters.
    """
    op_dict = {0: qml.RX, 1: qml.RY, 2: qml.RZ}
    dev = qml.device("default.qubit", wires=2)

    obs = qml.PauliZ(measured_wire)
    operator = op_dict[op_order[diff_idx]](params[diff_idx], wires[diff_idx])
    gen, coeff = generator_info(operator)
    '''
    @qml.qnode(dev)
    def circuit_bra1():

        # Put your code here #

        ######################
        return qml.state()

    @qml.qnode(dev)
    def circuit_ket1():

        # Put your code here #

        ######################
        return qml.state()

    @qml.qnode(dev)
    def circuit_bra2():

        # Put your code here #

        ######################
        return qml.state()

    @qml.qnode(dev)
    def circuit_ket2():

        # Put your code here #

        ######################
        return qml.state()

    bra1 = circuit_bra1()
    ket1 = circuit_ket1()
    bra2 = circuit_bra2()
    ket2 = circuit_ket2()
    '''
    '''testing the 2 side approach'''
    '''
    ops = [
        qml.RX(x[0], wires=0),
        qml.CNOT(wires=(0,1)),
        qml.RY(x[1], wires=1),
        qml.RZ(x[2], wires=1)
    ]
    M = qml.PauliX(wires=1)
    '''
    
    grads = []
    ops = []
    
    state = dev._create_basis_state(0)
    #bra = dev._create_basis_state(0)
    #ket = dev._create_basis_state(0)
    A = qml.PauliZ(wires=measured_wire)
    
    ## fill out ops[]
    for i in range(len(op_order)):
        ops.append(op_dict[op_order[i]](params[i], wires[i]))
    
    #print(ops)
    
    for op in ops:
        state = dev._apply_operation(state, op)
    
    print(state)
    
    bra = dev._apply_operation(state, A)
    ket = state
    
    ''' works below
    bra_loop = dev._apply_operation(state, A)
    ket_loop = state
    for op in reversed(ops):
        adj_op = qml.adjoint(op)
        bra_loop = dev._apply_operation(bra_loop, adj_op)
        ket_loop = dev._apply_operation(ket_loop, adj_op)
        print(np.vdot(bra_loop, ket_loop))
   ''' 
    
    ## create <bra|
    #for op in reversed(ops:
    #    bra = dev._apply_operation(bra, op)
    #bra = dev._apply_operation(bra, A)    
    
    ## Loops through |ket> building gradient list
    for op in reversed(ops):
        adj_op = qml.adjoint(op)
        ket = dev._apply_operation(ket, adj_op)

        # Calculating the derivative
        #if op.num_params != 0:
        dU = qml.operation.operation_derivative(op)

        ket_temp = dev._apply_unitary(ket, dU, op.wires)

        dM = 2 * np.real(np.vdot(bra, ket_temp))
        grads.append(dM)

        bra = dev._apply_operation(bra, adj_op)
    
    grads = grads[::-1]
    print(grads)
    ## return the diff_idx'th gradient
    return  grads[diff_idx] # Put your code here #


# These functions are responsible for testing the solution.

def run(test_case_input: str) -> str:
    op_order, params, diff_idx, wires, measured_wire = json.loads(test_case_input)
    params = np.array(params, requires_grad=True)
    der = derivative(op_order, params, diff_idx, wires, measured_wire)
    return str(der)

def check(solution_output: str, expected_output: str) -> None:
    solution_output = json.loads(solution_output)
    expected_output = json.loads(expected_output)
    assert np.allclose(
        solution_output, expected_output, rtol=1e-4
    ), "Your derivative isn't quite right!"


test_cases = [['[[1,0,2,1,0,1], [1.23, 4.56, 7.89, 1.23, 4.56, 7.89], 0, [1, 0, 1, 1, 1, 0], 1]', '-0.2840528']]

for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")

    try:
        output = run(input_)

    except Exception as exc:
        print(f"Runtime Error. {exc}")

    else:
        if message := check(output, expected_output):
            print(f"Wrong Answer. Have: '{output}'. Want: '{expected_output}'.")

        else:
            print("Correct!")

Running test case 0 with input '[[1,0,2,1,0,1], [1.23, 4.56, 7.89, 1.23, 4.56, 7.89], 0, [1, 0, 1, 1, 1, 0], 1]'...
[[0.08353534-0.06994494j 0.53578176-0.45144286j]
 [0.05615498+0.09267143j 0.36294834+0.59480579j]]
[-0.28405277861717027, 0.0, -0.10120785176380524, 0.04609728029065144, -0.000939685170580868, 6.917736238573884e-17]
Correct!
